<a href="https://colab.research.google.com/github/Mar3334/Patimetria/blob/main/Patimetria2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch==1.4.0 torchvision==0.5.0

     |████████████████████████████████| 753.4MB 21kB/s 
     |████████████████████████████████| 4.0MB 52.7MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [4]:
!cp -rf /content/drive/MyDrive/siamese-triplet /content/drive/MyDrive/siamese-triplet

cp: cannot copy a directory, '/content/drive/MyDrive/siamese-triplet', into itself, '/content/drive/MyDrive/siamese-triplet/siamese-triplet'


In [5]:
import torch
from torchvision import models

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/siamese-triplet')

In [7]:
from networks import VGGEmbeddingNet, EmbeddingNet, TripletNet
vgg_model = VGGEmbeddingNet()
emb_net = EmbeddingNet()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth


In [9]:
from duckies_dataset import DuckieDataset

In [10]:
from torchvision import transforms
from duckies_dataset import Rescale
trans = transforms.Compose([Rescale(224),
                            transforms.ToTensor()])

duckie_dataset = DuckieDataset('/content/drive/MyDrive/train', transform= trans)

In [11]:
from datasets import TripletMNIST
triplet_dataset = TripletMNIST(duckie_dataset)

<class 'torch.Tensor'>


In [12]:
from trainer import fit, train_epoch, test_epoch
from datasets import TripletMNIST
train_dataset = DuckieDataset("/content/drive/MyDrive/train", transform= trans)
test_dataset = DuckieDataset("/content/drive/MyDrive/train", train=False, transform= trans)

In [13]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable
from losses import TripletLoss
import numpy as np
cuda = torch.cuda.is_available()
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
# Set up data loaders

triplet_train_dataset = TripletMNIST(train_dataset) # Returns triplets of images
triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 32
kwargs = {'num_workers': 8, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(vgg_model)
if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 10
log_interval = 500

<class 'torch.Tensor'>


In [14]:
fit(triplet_train_loader, triplet_test_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/1080 (0%)]	Loss: 0.852113
Epoch: 1/10. Train set: Average loss: 0.7974
Epoch: 1/10. Validation set: Average loss: 0.6137
Train: [0/1080 (0%)]	Loss: 0.602837
Epoch: 2/10. Train set: Average loss: 0.5131
Epoch: 2/10. Validation set: Average loss: 0.4345
Train: [0/1080 (0%)]	Loss: 0.249085
Epoch: 3/10. Train set: Average loss: 0.4077
Epoch: 3/10. Validation set: Average loss: 0.3909
Train: [0/1080 (0%)]	Loss: 0.295516
Epoch: 4/10. Train set: Average loss: 0.4205
Epoch: 4/10. Validation set: Average loss: 0.3671
Train: [0/1080 (0%)]	Loss: 0.595594
Epoch: 5/10. Train set: Average loss: 0.5078
Epoch: 5/10. Validation set: Average loss: 0.4656
Train: [0/1080 (0%)]	Loss: 0.546804
Epoch: 6/10. Train set: Average loss: 0.4727
Epoch: 6/10. Validation set: Average loss: 0.6497
Train: [0/1080 (0%)]	Loss: 0.477820
Epoch: 7/10. Train set: Average loss: 0.4735
Epoch: 7/10. Validation set: Average loss: 0.3942
Train: [0/1080 (0%)]	Loss: 0.299744
Epoch: 8/10. Train set: Average loss: 0.3842
Ep

In [15]:
# Define functions to extract embeddings using diferent datasets
def extract_embeddings(dataset, model, dims = 1024, opt = None):
    with torch.no_grad():
        model.eval()
        embeddings = np.zeros((len(dataset), dims))
        labels = list()
        for k, it_ in enumerate(dataset):
            images = it_[0].unsqueeze_(0)
            images = images.cuda()
            target = it_[1]
            if opt is None:
                aux = model.get_embedding(images).data.cpu().numpy()
            else:
                aux = model.get_embedding(images, opt).data.cpu().numpy()
            embeddings[k] = aux.reshape(1, dims)
            labels.append(target)
    return embeddings, labels


In [16]:
embeddings, labels = extract_embeddings(test_dataset, vgg_model, dims = 512)

In [38]:
from PIL import Image

def n_closest_images(image, n=5):
  img = Image.open(image)
  img = trans(img)
  emb_img = vgg_model(img.unsqueeze(0).cuda())
  embedding_img = emb_img.data.cpu().numpy()
  list_dist, list_labels, neat_list, label_ubication_list = [], [], [], []
  for i in range(embeddings.shape[0]):
    emb2 = embeddings[i,:]
    dist = np.sum((emb2 - embedding_img)**2)
    list_dist.append(dist)
  neat_list[:] = list_dist
  neat_list.sort()
  for i in range(n):
    position = list_dist.index(neat_list[i])
    label_ubication = position - labels.index(labels[position])
    label_ubication_list.append(label_ubication)
    list_labels.append(labels[position])
  print (neat_list[:n], list_labels, label_ubication_list)

In [39]:
n_closest_images("/content/drive/MyDrive/train/unclesam/unclesamrubberduckback400x400.jpg")

[0.0, 0.004717058665921881, 0.02648389478763237, 0.02892731603291443, 0.03103558507423607] [tensor(281.), tensor(279.), tensor(279.), tensor(238.), tensor(238.)] [0, 2, 3, 1, 0]


In [25]:
test_dataset.class_to_idx

{'AlCapo': 0,
 'Alibaba': 1,
 'Americancop': 2,
 'Aquaman': 3,
 'Ariana': 4,
 'Artist': 5,
 'Astronaut': 6,
 'Avatara': 7,
 'Aztec': 8,
 'BabyBoyKeychain': 9,
 'Bali': 10,
 'Balifemale': 11,
 'BalletDancer': 12,
 'Balthazar': 13,
 'Basketball': 14,
 'Bat': 15,
 'Batmangreyholdy': 16,
 'BavarianMan': 17,
 'BavarianWoman': 18,
 'Bicycles': 19,
 'BigBen': 20,
 'Black': 21,
 'BlackSheep': 22,
 'BlackStar': 23,
 'Blacksheepmini': 24,
 'Blue': 25,
 'BlueSuede': 26,
 'Bluemini': 27,
 'BohemianQuacksody': 28,
 'Bombshell': 29,
 'Borntosun': 30,
 'Bowling': 31,
 'Boxer': 32,
 'Brit': 33,
 'Cactus': 34,
 'Caesar': 35,
 'Cannabis': 36,
 'CaptainQuack': 37,
 'Carnivalman': 38,
 'Carnivalwoman': 39,
 'Chef': 40,
 'Chewquacker': 41,
 'Chicken': 42,
 'ChimneySweep': 43,
 'Christmasjumper': 44,
 'Cleaningfairy': 45,
 'Clouds': 46,
 'Cop': 47,
 'Crocodile': 48,
 'Cyber': 49,
 'Cybersoldier': 50,
 'Dali': 51,
 'Dark': 52,
 'DarkDuckWoman': 53,
 'Delftblue': 54,
 'Diamont': 55,
 'Dino': 56,
 'Diva': 57,
